In [2]:
%load_ext autoreload
%autoreload 2

import pyVHR as vhr
import numpy as np
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.plot.visualize import *
import os
import plotly.express as px
from pyVHR.utils.errors import getErrors, printErrors, displayErrors

import constants
import pandas as pd

vhr.plot.VisualizeParams.renderer = 'vscode' 

In [3]:
# -- LOAD A DATASET

dataset_name = 'lgi_ppgi'    
video_DIR, BVP_DIR = constants.get_dataset_paths(dataset_name)

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

0 D:/datasets_rppg/lgi_ppgi\alex\alex_gym\cv_camera_sensor_stream_handler.avi
1 D:/datasets_rppg/lgi_ppgi\alex\alex_resting\cv_camera_sensor_stream_handler.avi
2 D:/datasets_rppg/lgi_ppgi\alex\alex_rotation\cv_camera_sensor_stream_handler.avi
3 D:/datasets_rppg/lgi_ppgi\alex\alex_talk\cv_camera_sensor_stream_handler.avi
4 D:/datasets_rppg/lgi_ppgi\angelo\angelo_gym\cv_camera_sensor_stream_handler.avi
5 D:/datasets_rppg/lgi_ppgi\angelo\angelo_resting\cv_camera_sensor_stream_handler.avi
6 D:/datasets_rppg/lgi_ppgi\angelo\angelo_rotation\cv_camera_sensor_stream_handler.avi
7 D:/datasets_rppg/lgi_ppgi\angelo\angelo_talk\cv_camera_sensor_stream_handler.avi
8 D:/datasets_rppg/lgi_ppgi\cpi\cpi_gym\cv_camera_sensor_stream_handler.avi
9 D:/datasets_rppg/lgi_ppgi\cpi\cpi_resting\cv_camera_sensor_stream_handler.avi
10 D:/datasets_rppg/lgi_ppgi\cpi\cpi_rotation\cv_camera_sensor_stream_handler.avi
11 D:/datasets_rppg/lgi_ppgi\cpi\cpi_talk\cv_camera_sensor_stream_handler.avi
12 D:/datasets_rppg/lgi_

In [4]:
# -- PARAMETER SETTING

wsize = 8        # seconds of video processed (with overlapping) for each estimate 
video_idx = 1    # index of the video to be processed
fname = dataset.getSigFilename(video_idx)
sigGT = dataset.readSigfile(fname)
bpmGT, timesGT = sigGT.getBPM(wsize)
videoFileName = dataset.getVideoFilename(video_idx)
print('Video processed name: ', videoFileName)
fps = vhr.extraction.get_fps(videoFileName)
print('Video frame rate:     ',fps)

Video processed name:  D:/datasets_rppg/lgi_ppgi\alex\alex_resting\cv_camera_sensor_stream_handler.avi
Video frame rate:      25.0


# Compare by landmark

In [21]:
import pyVHR.analysis.pipelineLandmarks as custom_pipeline

path_results = "../results/test_landmarks/h5/" # general path for cfg# general path for data
filenameH5 = path_results + "alex_each_landmark" + ".h5"

def analyze_landmark(dataset, approach, metrics=['MAE'], j_data=False, writeFile=False):
    """Print boxplot analysis for each landmark by metric"""
    print('   filename h5: ', filenameH5)
    # boxplot
    st_lmk, figures = [], []
    for m in metrics:
        st = custom_pipeline.StatAnalysisLdmk(filenameH5, join_data=j_data)
        if m=='MAE': 
            scale = 'log' 
        else: 
            scale = 'linear'
        fig = st.displayBoxPlot(metric=m, scale=scale)
        figures.append(fig)
        st_lmk.append(st)
    return st_lmk, figures

st_lmk, figures = analyze_landmark(dataset='LGI_PPGI', approach='patches', metrics=['MAE','PCC'], j_data=False, writeFile=False)

   filename h5:  ../results/test_landmarks/h5/alex_each_landmark.h5


In [22]:
fig = figures[0]
st = st_lmk[0]
metric = 'MAE'
tit = "Metric: " + metric
top = 30 # 40

fig.update_layout(
    width=1300,
    height=500,
    title=tit,
    yaxis_type=st.scale[metric],
    xaxis_type="category",
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        # dtick=gwidth,
        gridcolor='rgb(255,255,255)',
        gridwidth=.1,
        zerolinewidth=2,
        title_text=metric,
        titlefont=dict(size=30)
    ),
    font=dict( 
        family="monospace",
        size=8,
        color='black'
    ),
    showlegend=False
)
# fig.update_xaxes(tickangle=90)
fig

In [23]:
fig = figures[1]
st = st_lmk[1]
metric = 'PCC'
tit = "Metric: " + metric
top = 30 # 40

fig.update_layout(
    width=1300,
    height=500,
    title=tit,
    yaxis_type=st.scale[metric],
    xaxis_type="category",
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        # dtick=gwidth,
        gridcolor='rgb(255,255,255)',
        gridwidth=.1,
        zerolinewidth=2,
        title_text=metric,
        titlefont=dict(size=30)
    ),
    font=dict(
        family="monospace",
        size=8,
        color='black'
    ),
    showlegend=False
)
# fig.update_xaxes(tickangle=90)
fig

In [24]:
data = st_lmk[0].get_data()
methods = st.dataFrame[0].method.unique()
data = pd.DataFrame(data.values(), columns=methods, index=data.keys())
data['Median'] = data.median(axis=1)
data = data.sort_values(['Median'])
data

,CHROM,POS,LGI,OMIT,Median
right_lower_lateral_forehead,0.977228,0.910452,1.066865,1.273733,1.022047
glabella,1.077370,1.020375,1.107111,1.687618,1.092241
lower_medial_forehead,0.998461,0.958333,1.245990,1.973944,1.122226
left_lower_lateral_forehead,1.178489,1.140388,1.153306,2.400420,1.165897
upper_nasal_dorsum,1.622248,1.687411,1.767327,2.032434,1.727369
right_marionette_fold,0.950994,1.336145,2.202859,2.278912,1.769502
nasal_tip,1.644191,1.914284,1.678459,3.833896,1.796372
right_malar,1.308031,1.594076,2.028306,2.731416,1.811191
chin,1.814719,1.639530,1.963986,3.058475,1.889353
right_lower_cheek,1.023201,1.095733,5.144147,2.802320,1.949026


# Compare by video

TODO groupby and best 5 landamrks for all videos

In [28]:
path_results = "../results/test_landmarks/h5/" # general path for cfg# general path for data
filenameH5 = path_results + f"LGI_PPGI_resting_each_landmark.h5"
dataset = 'LGI_PPGI'
metric = 'PCC'
method = 'median'

# Get unique list values in df column
def unique_list_values(df, column_name):
    seen_values = set()

    for row in df[column_name]:
        if tuple(row) not in seen_values:
            seen_values.add(tuple(row))

    return [list(x) for x in seen_values]

def DETAILS(dataset, filenameH5, metric='MAE'):
  """Show performance details on each video of the dataset"""

  print('## Dataset: ' + dataset)
  print('   filename h5: ', filenameH5)
  df = pd.read_hdf(filenameH5)
  df = df.sort_values(metric)
  datasetname = df['dataset'][0].upper()
  landmarks = unique_list_values(df, 'landmarks')

  # # loop on landmarks
  for ldmk in landmarks:
    print('   landmark     :', ldmk)
    df_ldmk = df[df['landmarks'].apply(lambda x: x == ldmk)]
    vals = df_ldmk[metric]
    fvideos = df_ldmk['videoFilename']
    x = []
    y = []
    for i in vals.index:
      n = fvideos[i]
      # n = n[n.rfind('/', 0, n.rfind('/'))+1:n.rfind('/')]
      n = n[n.rfind('\\', 0, n.rfind('\\'))+1:n.rfind('\\')]
      x.append(n)
      y.append(vals[i][0])
    fig = px.bar(x=x, y=y, title='dataset: ' + datasetname + ' landmarks: ' + str(ldmk))
    fig.update_xaxes(type='category')
    fig.show()

DETAILS(dataset, filenameH5, metric='PCC')

## Dataset: LGI_PPGI
   filename h5:  ../results/test_landmarks/h5/LGI_PPGI_resting_each_landmark.h5
   landmark     : ['right_temporal']


   landmark     : ['left_marionette_fold']


   landmark     : ['right_malar']


   landmark     : ['left_mid_nasal_sidewall']


   landmark     : ['chin']


...

   landmark     : ['glabella']


   landmark     : ['nasal_tip']


   landmark     : ['right_upper_lip']


   landmark     : ['right_nasolabial_fold']


   landmark     : ['lower_medial_forehead']
